In [1]:
from Metrics import *
from TimewiseKfold import *
from InteractionDetection import *
from sklearn import linear_model
import numpy as np

In [2]:
scaled_folds, PROVINCES_NUM = splitting_data(is_scaled=True, region=2)
ndvi_folds, PROVINCES_NUM = splitting_data(is_NDVI=True, region=2)

In [3]:
selected_features = ['Rdays_VII', 'Rdays_X', 'Srad_I', 'Srad_IX', 'Srad_X']

#1st iteration
selected_features += ['Eva_I', 'Rdays_III', 'Tmax_avg_IX', 'Tmin_II']

#2st iteration
selected_features += ['Eva_IV']

#3rd iteration
selected_features += ['Prcp_max_XII']

#4th iteration
selected_features += ['Prcp_X']

alphas_res = [
   np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0. , 0.5, 0. , 0.5, 0.5]), 
   np.array([0.5, 0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0. , 0.5, 0.5, 0.5, 0.5]), 
   np.array([0.5, 0. , 0.5, 0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]), 
   np.array([0.5, 0.5, 0.5, 0.5, 0. , 0.5, 0.5, 0.5, 0.5, 1. , 0.5, 0.5]), 
   np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 0. , 0.5, 0.5]), 
   np.array([0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 0.5, 0.5]), 
   np.array([0.5, 1. , 0.5, 0.5, 0.5, 0.5, 0. , 0.5, 0.5, 0.5, 0.5, 0.5]) 
]
deltas_res = [
   np.array([0., 0., 0., 0., 1., 0.]), 
   np.array([0., 0., 1., 0., 0., 0.]), 
   np.array([0., 0., 0., 1., 0., 0.]), 
   np.array([1., 0., 0., 0., 0., 0.]), 
   np.array([0., 0., 0., 0., 1., 0.]), 
   np.array([0., 0., 0., 0., 1., 0.]), 
   np.array([0., 1., 0., 0., 0., 0.])
]

transformed_folds = transform_data(scaled_folds=scaled_folds, selected_features=selected_features, alphas=alphas_res, deltas=deltas_res)

In [4]:
selected_ndvi = ['NDVI_I', 'NDVI_II', 'NDVI_III', 'NDVI_IV', 'NDVI_V', 'NDVI_VI', 'NDVI_VII', 'NDVI_VIII', 'NDVI_IX', 'NDVI_X', 'NDVI_XI', 'NDVI_XII']
# selected_ndvi = ['NDVI_V', 'NDVI_X']

In [5]:
def combine_ndvi(X_train, X_test, order):
    ndvi_train, ndvi_test, y_train, y_test = split_train_test(fold=ndvi_folds[order])
    
    ndvi_train_reduced = ndvi_train[selected_ndvi].to_numpy()
    ndvi_test_reduced = ndvi_test[selected_ndvi].to_numpy()
    
    for i in range(0, len(X_train)):
        X_train[i] += list(ndvi_train_reduced[i])
    
    for i in range(0, len(X_test)):
        X_test[i] += list(ndvi_test_reduced[i])
        
    return X_train, X_test


In [6]:
table = metrics_table()

for (i, fold) in enumerate(transformed_folds):
    X_train, X_test, y_train, y_test = split_train_test(fold=fold)
    X_train, X_test = combine_ndvi(X_train, X_test, order=i)
    
    linear_model = LinearRegression(n_jobs=-1).fit(X=X_train, y=y_train)
    
    row = [MAXIMUM_YEAR - FOLD_NUM + 1 + i]
    
    row += metrics_calculation(y_actual=y_test, y_pred=linear_model.predict(X_test), y_train=y_train)
    
    table.loc[len(table)] = row
    
    del linear_model

table
    

,Year,RMSE,R2_Score,MSE,MAE,RAE,RSE
0,2018.0,1.206565,0.507076,1.455799,0.863518,0.602341,0.536295
1,2019.0,0.925768,0.461076,0.857046,0.781602,0.756639,0.541772
2,2020.0,5.719031,0.088835,32.707319,3.830478,1.031269,1.277286
